<a href="https://colab.research.google.com/github/pakdaniel/DataMiningFinal/blob/main/DataMiningFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install pysofaconventions
!pip3 install colorama

     |████████████████████████████████| 89 kB 5.9 MB/s 


In [7]:
import tensorflow as tf
import numpy as np 
import random
import matplotlib.pyplot as plt
import os 
import sys
import librosa
import pysofaconventions 
import colorama
from warnings import warn

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [9]:
from google.colab import drive
drive_root = '/content/drive'
drive.mount(drive_root)

HUTUBS_PATH = os.path.join(drive_root, "My Drive", "HUTUBS")
HRIR_DIR = os.path.join(HUTUBS_PATH, "HRIRs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
class SOFA:
  def __init__(self, obj):

    self.IR = obj.getDataIR().data
    self.SamplingRate = obj.getSamplingRate().data.item()
    self.SamplingRateUnits = obj.getSamplingRateUnits()
    self.Delay = obj.getDataDelay().data
      
    dim_dict = obj.getDimensionsAsDict()
    self.M = dim_dict["M"].size # Num measurement positions
    self.R = dim_dict["R"].size # Num receivers (ears)
    self.E = dim_dict["E"].size # Num emitters (speakers)
    self.N = dim_dict["N"].size # Length of HRTF
    self.I = dim_dict["I"].size 
    self.S = dim_dict["S"].size

    self.Listener = {
        "Position": obj.getListenerPositionValues().data,
        "PositionType": obj.getListenerPositionInfo()[1],
        "PositionUnits": obj.getListenerPositionInfo()[0],
        "Up": obj.getListenerUpValues().data,
        "View": obj.getListenerViewValues().data,
        "ViewType": obj.getListenerViewInfo()[1],
        "ViewUnits": obj.getListenerViewInfo()[0],
    }

    self.Receiver = {
        "Position": obj.getReceiverPositionValues().data,
        "PositionType": obj.getReceiverPositionInfo()[1],
        "PositionUnits": obj.getReceiverPositionInfo()[0],
    }

    self.Source = {
        "Position": obj.getSourcePositionValues().data,
        "PositionType": obj.getSourcePositionInfo()[1],
        "PositionUnits": obj.getSourcePositionInfo()[0],
    }

    self.Emitter = {
        "Position": obj.getEmitterPositionValues().data,
        "PositionType": obj.getEmitterPositionInfo()[1],
        "PositionUnits": obj.getEmitterPositionInfo()[0],
    }

  @staticmethod
  def sph2cart(coords):
      alpha = coords[:, 0]*(np.pi/180)
      beta = coords[:, 1]*(np.pi/180)
      r = coords[:, 2]

      x = r * np.cos(alpha) * np.cos(beta)
      y = r * np.sin(alpha) * np.cos(beta)
      z = r * np.sin(beta)
      return np.array([x, y, z]).T

  @staticmethod
  def cart2sph(coords):
      x = coords[:, 0]
      y = coords[:, 1]
      z = coords[:, 2]
      
      r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
      alpha = np.arctan2(y, x)
      beta = np.arcsin(z / np.where(r != 0, r, 1))
      return alpha, beta, r

def read_sofa(sofa_path):
  return SOFA(pysofaconventions.SOFAFile(sofa_path,'r'))